<a href="https://colab.research.google.com/github/joaochenriques/MCTE_2020_2021/blob/main/DiskActuator/CT_Newton_Raphson.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
from sympy import *
import mpmath as mp
init_printing()

In [32]:
Eq.__getitem__ = lambda self, i: self.args[i]
# To allow equation.subs( [Eq(x,y)] )

# Define the variables of the problem

In [33]:
ζ1, ζ4, ζ4t, ζ4b, ζ5 = symbols( r"\zeta_1, \zeta_4, \zeta_{4t}, \zeta_{4b}, \zeta_5" )

In [34]:
Fr1, Fr4, Fr4t, Fr4b, Fr5 = symbols( r"\mathrm{Fr}_1, \mathrm{Fr}_4, \mathrm{Fr}_{4t}, \mathrm{Fr}_{4b}, \mathrm{Fr}_5" )

In [35]:
B = symbols( "B")
nt = symbols( "n_t")

In [ ]:
C_T, C_P = symbols( "C_\mathrm{T}, C_\mathrm{P}")
Fr2t = symbols( "\mathrm{Fr}_{2t}")

# Data of the problem

In [160]:
data = { B: 0.3, ζ1: 1, Fr1: 0.15, C_T: 1.0 }
data

# Define the equations required to compute the conditions at (4)

In [161]:
EqA1 = Eq( ζ4, ζ1 + Fr1**2/2 - Fr4b**2/2 )
EqA1

In [162]:
EqA2 = Eq( Fr4t * ζ4t + Fr4b * ζ4b, Fr1 * ζ1 )
EqA2

In [163]:
EqA3 = Eq( ζ4, ζ4t + ζ4b )
EqA3

In [164]:
EqA4 = Eq( Fr4t**2 * ζ4t + Fr4b**2 * ζ4b - Fr1**2 * ζ1, -( ζ4**2 - ζ1**2 )/2 - B/2*C_T*Fr1**2 )
EqA4

In [165]:
sys01 = [ EqA1.subs(data).lhs-EqA1.subs(data).rhs, 
          EqA2.subs(data).lhs-EqA2.subs(data).rhs, 
          EqA3.subs(data).lhs-EqA3.subs(data).rhs, 
          EqA4.subs(data).lhs-EqA4.subs(data).rhs,
          EqCT.subs(data).lhs-EqCT.subs(data).rhs
        ]
sys1 = Matrix( sys01 )
sys1

⎡                                      2                                      
⎢                      \mathrm{Fr}_{4b}                                       
⎢                      ───────────────── + \zeta₄ - 1.01125                   
⎢                              2                                              
⎢                                                                             
⎢        \mathrm{Fr}_{4b}⋅\zeta_{4b} + \mathrm{Fr}_{4t}⋅\zeta_{4t} - 0.15     
⎢                                                                             
⎢                        \zeta₄ - \zeta_{4b} - \zeta_{4t}                     
⎢                                                                             
⎢                                                                    2        
⎢                2                              2              \zeta₄         
⎢\mathrm{Fr}_{4b} ⋅\zeta_{4b} + \mathrm{Fr}_{4t} ⋅\zeta_{4t} + ─────── - 0.519
⎢                                                   

In [166]:
vars1 = Matrix( [ ζ4b, ζ4t, ζ4, Fr4t, Fr4b ] )
J1 = sys1.jacobian( vars1 )
J1

⎡        0                  0            1                     0              
⎢                                                                             
⎢\mathrm{Fr}_{4b}   \mathrm{Fr}_{4t}     0                \zeta_{4t}          
⎢                                                                             
⎢       -1                 -1            1                     0              
⎢                                                                             
⎢                2                  2                                         
⎢\mathrm{Fr}_{4b}   \mathrm{Fr}_{4t}   \zeta₄    2⋅\mathrm{Fr}_{4t}⋅\zeta_{4t}
⎢                                                                             
⎣        0                  0            0     88.8888888888889⋅\mathrm{Fr}_{4

             \mathrm{Fr}_{4b}         ⎤
                                      ⎥
                \zeta_{4b}            ⎥
                                      ⎥
                    0                 ⎥
         

In [167]:
b = -sys1
b

⎡                                        2                                    
⎢                        \mathrm{Fr}_{4b}                                     
⎢                      - ───────────────── - \zeta₄ + 1.01125                 
⎢                                2                                            
⎢                                                                             
⎢        -\mathrm{Fr}_{4b}⋅\zeta_{4b} - \mathrm{Fr}_{4t}⋅\zeta_{4t} + 0.15    
⎢                                                                             
⎢                        -\zeta₄ + \zeta_{4b} + \zeta_{4t}                    
⎢                                                                             
⎢                                                                      2      
⎢                  2                              2              \zeta₄       
⎢- \mathrm{Fr}_{4b} ⋅\zeta_{4b} - \mathrm{Fr}_{4t} ⋅\zeta_{4t} - ─────── + 0.5
⎢                                                   

In [172]:
values1 = Matrix( [ 0.99, 0.25, 0.73, 0.13, 0.20 ] )

for i in range( 20 ):
  vv1 = dict( zip( vars1, values1 ) )
  BB = b.subs( vv1 )
  JJ = J1.subs( vv1 )
  delta = JJ.LUsolve( BB )
  values1 += delta
  if delta.norm() < 1E-14: 
    break
    
delta.norm() 

In [177]:
M1 = symbols( "M_{1}" )
M1_value = Eq( M1, Fr1**2 * ζ1 ).subs( data ).subs( sol1 )
M1_value

In [178]:
M4 = symbols( "M_{4}" )
M4_value = Eq( M4, Fr4t**2 * ζ4t + Fr4b**2 * ζ4b ).subs( data ).subs( sol1 )
M4_value

# Compute coefficients

In [180]:
EqFr2t.subs( data ).subs( sol1 )

In [181]:
EqCP.subs( [EqFr2t] ).subs( [EqCT] ).subs( data ).subs( sol1 )

# Compute conditions at section (5)

Mass balance

In [182]:
eqB1 = Eq( Fr1 * ζ1, Fr5 * ζ5 )
eqB1

Momentum balance

In [183]:
eqB2 = Eq( Fr5**2 * ζ5 - ( Fr4b**2 * ζ4b + Fr4t**2 * ζ4t ), -( ζ5**2 - ζ4**2 )/2 )
eqB2

In [185]:
sys2 = [ eqB1.subs(data).subs( sol1 ), eqB2.subs(data).subs( sol1 ) ]
sys2

In [186]:
sys2[0]

In [187]:
Eq( expand( sys2[1].lhs*ζ5 ), expand( sys2[1].rhs*ζ5 ) ) 

In [188]:
sol2 = solve( sys2, [ ζ5, Fr5 ], dict=True )

In [189]:
print( "Number of sets of solutions: ", len( sol2 ) )

Number of sets of solutions:  3


In [190]:
sol2[0] 

In [191]:
sol2[1]

In [192]:
sol2[2]

In [193]:
Δζ15 = symbols( r"\Delta\zeta_{15}" )
eqC1 = Eq( Δζ15, ζ1 - sol2[1][ζ5] ).subs( data )
eqC1

In [195]:
Δζ45 = symbols( r"\Delta\zeta_{45}" )
eqD1 = Eq( Δζ45, round( sol1[ζ4] - re( sol2[1][ζ5] ), 4 ) )
eqD1

In [196]:
M5 = symbols( "M_5" )
M5_value = simplify( Eq( M5, Fr5**2 * ζ5 ).subs( sol2[1] ) )
M5_value 